# Chapter 7. 텍스트 문서의 범주화 - (5) CNN 모델로 문서 요약하기

- 리뷰는 많은 문장으로 구성돼 있으며, 일부를 통해 전체 문서의 성향을 결정할 수 있다.
- 첫 번째 단계는 각 문장에 중요도 점수를 할당해 문서의 돌출맵(saliency map)을 만드는 것이다.
- 문서 요약 과정
    1. 네트워크의 전방 전달을 통해 문서에 대한 클래스 예측을 생성한다.
    2. 네트워크의 예측을 뒤집어서 의사 레이블(pseudo label)을 만든다.
    3. 의사 레이블을 실제 레이블처럼 손실 함수에 제공하여 강한 긍정 문장일 수록 높은 기울기 노름을 보이도록 한다.
    4. 문장 임베딩층에 손실 함수의 미분을 계산한다.
    5. 가장 중요한 문장이 위에 가도록 내림차순 기울기 노름으로 문장을 정렬한다.
    

In [1]:
import config
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import keras.backend as K
from model.cnn_document_model import DocumentModel
from preprocessing.utils import Preprocess, remove_empty_docs
from dataloader.loader import Loader

Using TensorFlow backend.


## 학습된 모델 로드하기

In [2]:
# imdb 전체 데이터셋으로 미리 학습한 CNN 모델 로드
imdb_model = DocumentModel.load_model(config.MODEL_DIR+ '/imdb/full_model_10.json')
imdb_model.load_model_weights(config.MODEL_DIR+ '/imdb/full_model_10.hdf5')

model = imdb_model.get_classification_model()
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=["accuracy"])


Vocab Size = 28656  and the index of vocabulary words passed has 28654 words
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 데이터셋 로드하기

In [3]:
# imdb 학습셋 로드
train_df = Loader.load_imdb_data(directory='train')
print(f'train_df.shape : {train_df.shape}')

corpus = train_df['review'].tolist()
target = train_df['sentiment'].tolist()
corpus, target = remove_empty_docs(corpus, target)
print(f'corpus size : {len(corpus)}')
print(f'target size : {len(target)}')

train_df.shape : (25000, 2)
corpus size : 25000
target size : 25000


## 데이터셋을 인덱스 시퀀스로 변환

In [4]:
Preprocess.NUM_SENTENCES = 20
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

x_train = np.array(corpus_to_seq)
y_train = np.array(target)
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')

Found 28654 unique tokens.
All documents processed.x_train.shape : (25000, 600)
y_train.shape : (25000,)


## 의사 레이블 생성
모델이 학습셋에 대해 예측한 결과들을 각각 1에서 뺀 값을 구한다. 이것을 의사 레이블이라 한다

In [5]:
# 학습셋을 입력하여 예측결과들을 얻는다
preds = model.predict(x_train)
print(f'preds size : {len(preds)}')

# 예측결과를 1에서 뺀 값(의사 레이블)들을 얻는다
pseudo_label = np.subtract(1,preds)
print(f'pseudo_label size : {len(pseudo_label)}')

preds size : 25000
pseudo_label size : 25000


In [6]:
print(preds[:5])

[[0.56517434]
 [0.1569992 ]
 [0.80936414]
 [0.90274227]
 [0.0303922 ]]


In [7]:
print(pseudo_label[:5])

[[0.43482566]
 [0.84300077]
 [0.19063586]
 [0.09725773]
 [0.96960783]]


## 그래디언트 계산 함수 정의
keras의 model.optimizer.get_gradients() 함수를 이용하여 그래디언트(기울기)를 계산하는 함수를 정의한다

In [8]:
# # 학습된 문장 임배딩을 가져온다.(학습셋이 입력될 때 임베딩 레이어에서의 출력 벡터값)
sentence_ebd = imdb_model.get_sentence_model().predict(x_train)

input_tensors = [model.inputs[0],          # 입력 데이터
                 model.sample_weights[0],  # 샘플당 가중치
                 model.targets[0],         # 레이블                
                ]

# 문장 임베딩층의 텐서 변수
weights = imdb_model.get_sentence_model().outputs

# 전체 모델의 손실에 대하여 기울기 계산 (문장 임베딩층의 변수)
gradients = model.optimizer.get_gradients(model.total_loss, weights) 
get_gradients = K.function(inputs=input_tensors, outputs=gradients)   # 그래디언트 계산 함수

Instructions for updating:
Use tf.cast instead.


In [9]:
# 그래디언트 계산식의 입력 데이터
input_tensors

[<tf.Tensor 'input_1:0' shape=(?, 600) dtype=int32>,
 <tf.Tensor 'final_sample_weights:0' shape=(?,) dtype=float32>,
 <tf.Tensor 'final_target:0' shape=(?, ?) dtype=float32>]

In [10]:
# 그래디언트 계산식 입력의 샘플당 가중치
weights

[<tf.Tensor 'sentence_embeddings/transpose:0' shape=(?, 20, 90) dtype=float32>]

In [11]:
# 그래디언트 계산식의 입력 레이블
gradients

[<tf.Tensor 'gradients/sentence_conv/convolution/ExpandDims_grad/Reshape:0' shape=(?, 20, 90) dtype=float32>]

## 그래디언트 계산
위에서 정의한 계산함수를 이용하여 특정 문서의 그래디언트를 계산해보자. 입력 데이터가 들어오면 실제 예측값이 아닌, 의사레이블을 출력하도록 그래디언트를 계산한다. 아래와 같이 학습셋의 11번째 텍스트에 대하여 수행한다

In [12]:
document_number = 10                             # 학습셋 문서 중 n번째 문서에 대해
K.set_learning_phase(0)
inputs = [[x_train[document_number]],            # X
          [1],                                   # 샘플당 가중치
          [[pseudo_label[document_number][0]]],  # y (실제 레이블이 아닌 의사 레이블)
         ] 

grad = get_gradients(inputs)
print(f'grad[0].shape : {grad[0].shape}')

grad[0].shape : (1, 20, 90)


In [13]:
# n번째 문서 입력에 대한 그래디언트 출력값
grad

[array([[[ 1.1949699e-02, -1.3650034e-03,  3.4827441e-02, ...,
           9.0038568e-02,  5.3074103e-02,  6.3555222e-03],
         [-1.6439449e-02, -8.2099019e-03,  6.1936952e-02, ...,
           1.0885944e-01,  7.8125015e-02,  2.5040932e-02],
         [-5.1050939e-02, -2.8638870e-02,  1.9982873e-02, ...,
          -2.5617983e-02,  1.5553975e-03,  2.5209187e-02],
         ...,
         [ 6.8549318e-03, -1.3568846e-02, -1.3994600e-02, ...,
           1.6872341e-02, -1.3355829e-02, -1.2049110e-02],
         [-1.5267859e-04, -9.3831004e-05,  8.1380067e-06, ...,
           2.7823413e-04,  1.1438751e-04,  6.2266350e-05],
         [-5.7300007e-05, -2.6037883e-06, -1.3170496e-04, ...,
          -2.0489588e-06,  1.4252918e-04, -4.9144979e-05]]], dtype=float32)]

## 그래디언트 노름 값에 따른 문장 정렬
nltk.sent_tokenize로 문장을 구분한 뒤, 각 문장의 그래디언트 노름 값을 구하여 노름 값 기준 정렬을 수행한다

In [14]:
# 그래디언트 노름 값 추출하기
sent_score = []
sentences = sent_tokenize(corpus[document_number])  # 원본 문서를 문장 단위 split

# 각 문장의 노름 계산 (zero padding된 문장 빼고)
for i in range(min(len(sentences), Preprocess.NUM_SENTENCES)):
    sent_score.append((i, -np.linalg.norm(grad[0][0][i])))
print(f'sent_score size : {len(sent_score)}')

# 그래디언트 노름 값 기준으로 문장 인덱스들 정렬
sent_score.sort(key=lambda tup: tup[1])
summary_sentences = [ i for i, s in sent_score[:4]]
print('summary_sentences :', summary_sentences)

sent_score size : 17
summary_sentences : [13, 1, 12, 0]


In [15]:
doc = corpus[document_number]
label = y_train[document_number]
prediction = preds[document_number]
print(doc, label , prediction)

So it starts with a beautiful old house in the country. You have a group of people who get asked to come to this house and (not surprisingly) the caretakers always lock the gates at night for no apparent reason. Anywhoo, the people laugh, joke etc. This Dr tells them a spooky story of this woman and some kids. They get scared, they start to feel stuff. Oh no, a girl see's s ghost. Some more talking then this huge ghost comes and etc etc. This girl finds out that this ghost killed little kids and that she must free their souls, yeah yeah, blah blah. She does but, oh no, she dies as she does. And goes to heaven whilst this evil ghost goes to hell. Two people survive and escape the house. The script is terrible because a guy gets his head chopped off and Elanor (the one who dies saving the kids) says "oh no". The acting is wooden, the effects are crap and the set is a couple off rooms used over and over again. Basically if you like laughing at badly made films watch it, but if your lookin

In [17]:
# 요약한 문장들
for i in summary_sentences:
    print(i, sentences[i])

13 Basically if you like laughing at badly made films watch it, but if your looking for a scare then definitely give this film a miss.
1 You have a group of people who get asked to come to this house and (not surprisingly) the caretakers always lock the gates at night for no apparent reason.
12 The acting is wooden, the effects are crap and the set is a couple off rooms used over and over again.
0 So it starts with a beautiful old house in the country.


## 부정적인 리뷰 요약

In [18]:
document_number = [(i, a) for i, a in enumerate(corpus) 
                   if 'Wow, what a great cast! Julia' in a][0][0]
document_number

1890

In [19]:
K.set_learning_phase(0)
inputs = [[x_train[document_number]],            # X
          [1],                                   # 샘플당 가중치
          [[pseudo_label[document_number][0]]],  # y (실제 레이블이 아닌 의사 레이블)
         ] 
grad = get_gradients(inputs)

# 그래디언트 노름 값 추출하기
sent_score = []
sentences = sent_tokenize(corpus[document_number])  # 원본 문서를 문장 단위 split

# 각 문장의 노름 계산 (zero padding된 문장 빼고)
for i in range(min(len(sentences), Preprocess.NUM_SENTENCES)):
    sent_score.append((i, -np.linalg.norm(grad[0][0][i])))

# 그래디언트 노름 값 기준으로 문장 인덱스들 정렬
sent_score.sort(key=lambda tup: tup[1])
print(f'sent_score size : {len(sent_score)}')

sent_score size : 13


In [20]:
# 원본 문서
corpus[document_number]

'Wow, what a great cast! Julia Roberts, John Cusack, Christopher Walken, Catherine Zeta-Jones, Hank Azaria...what\'s that? A script, you say? Now you\'re just being greedy! Surely such a charismatic bunch of thespians will weave such fetching tapestries of cinematic wonder that a script will be unnecessary? You\'d think so, but no. America\'s Sweethearts is one missed opportunity after another. It\'s like everyone involved woke up before each day\'s writing/shooting/editing and though "You know what? I\'ve been working pretty hard lately, and this is guaranteed to be a hit with all these big names, right? I\'m just gonna cruise along and let somebody else carry the can." So much potential, yet so painful to sit through. There isn\'t a single aspect of this thing that doesn\'t suck. Even Julia\'s fat suit is lame.'

In [21]:
# 요약 문장들
for idx, score in sent_score[:6]:
    print(f'{idx} > {sentences[idx]}')

8 > I've been working pretty hard lately, and this is guaranteed to be a hit with all these big names, right?
12 > Even Julia's fat suit is lame.
5 > You'd think so, but no.
1 > Julia Roberts, John Cusack, Christopher Walken, Catherine Zeta-Jones, Hank Azaria...what's that?
7 > It's like everyone involved woke up before each day's writing/shooting/editing and though "You know what?
9 > I'm just gonna cruise along and let somebody else carry the can."


## 긍정적인 리뷰 요약

In [22]:
document_number = [(i, a) for i, a in enumerate(corpus) 
                   if 'This is what I was expecting when star trek DS9 premiered.' in a][0][0]
document_number

18792

In [23]:
K.set_learning_phase(0)
inputs = [[x_train[document_number]],            # X
          [1],                                   # 샘플당 가중치
          [[pseudo_label[document_number][0]]],  # y (실제 레이블이 아닌 의사 레이블)
         ] 
grad = get_gradients(inputs)

# 그래디언트 노름 값 추출하기
sent_score = []
sentences = sent_tokenize(corpus[document_number])  # 원본 문서를 문장 단위 split

# 각 문장의 노름 계산 (zero padding된 문장 빼고)
for i in range(min(len(sentences), Preprocess.NUM_SENTENCES)):
    sent_score.append((i, -np.linalg.norm(grad[0][0][i])))

# 그래디언트 노름 값 기준으로 문장 인덱스들 정렬
sent_score.sort(key=lambda tup: tup[1])
print(f'sent_score size : {len(sent_score)}')

sent_score size : 20


In [24]:
# 원본 문서
corpus[document_number]

"This is what I was expecting when star trek DS9 premiered. Not to slight DS9. That was a wonderful show in it's own right, however it never really gave the fans more of what they wanted. Enterprise is that show. While having a similarity to the original trek it differs enough to be original in it's own ways. It makes the ideas of exploration exciting to us again. And that was one of the primary ingredients that made the original so loved. Another ingredient to success was the relationships that evolved between the crew members. Viewers really cared deeply for the crew. Enterprise has much promise in this area as well. The chemistry between Bakula and Blalock seems very promising. While sexual tension in a show can often become a crutch, I feel the tensions on enterprise can lead to much more and say alot more than is typical. I think when we deal with such grand scale characters of different races or species even, we get some very interesting ideas and television. Also, we should note

In [27]:
# 요약 문장들
for idx, score in sent_score[-5:]:
    print(f'{idx} > {sentences[idx]}')

3 > Enterprise is that show.
2 > That was a wonderful show in it's own right, however it never really gave the fans more of what they wanted.
6 > And that was one of the primary ingredients that made the original so loved.
19 > Doesn't have to be orchestral.
0 > This is what I was expecting when star trek DS9 premiered.
